# Deep Convolutional and LSTM Recurrent Neural Networks for Multimodal Wearable Activity Recognition, Sensors 2016

One of the models considered in MLPerf DeepTS [proposal](http://bit.ly/2GIUcb3). Paper is [here](https://www.mdpi.com/1424-8220/16/1/115/htm). Code is [here](https://github.com/sussexwearlab/DeepConvLSTM).

![Neural network model](https://res.mdpi.com/sensors/sensors-16-00115/article_deploy/html/images/sensors-16-00115-g003.png)

In [1]:
import contextlib
import os, sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns.model import Model
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, LSTM, Activation, Dense, Reshape

In [2]:
# Study this notebook for details:
#   https://github.com/sussexwearlab/DeepConvLSTM/blob/master/DeepConvLSTM.ipynb
NB_SENSOR_CHANNELS = 113
NUM_CLASSES = 18
SLIDING_WINDOW_LENGTH = 24
FINAL_SEQUENCE_LENGTH = 8
BATCH_SIZE = 100
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128

In [3]:
# Seems like they describe exactly this model here:
#     https://stackoverflow.com/questions/53144762/convert-lasagne-to-keras-code-cnn-lstm
class DeepConvLSTMModel(Model):
    def __init__(self):
        super().__init__('DeepConvLSTMModel')

    def create(self):
        return models.Sequential([
            Model.Input((SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS, 1)),                   # (24, 113, 1)
            Conv2D(NUM_FILTERS, (FILTER_SIZE, 1), activation='relu', name='conv1/5x1'),    # (20, 113, 64)
            Conv2D(NUM_FILTERS, (FILTER_SIZE, 1), activation='relu', name='conv2/5x1'),    # (16, 113, 64)
            Conv2D(NUM_FILTERS, (FILTER_SIZE, 1), activation='relu', name='conv3/5x1'),    # (12, 113, 64)
            Conv2D(NUM_FILTERS, (FILTER_SIZE, 1), activation='relu', name='conv4/5x1'),    # (8,  113, 64)
            Reshape((FINAL_SEQUENCE_LENGTH, 113*64)),                                      # (8,  113*64)
            LSTM(NUM_UNITS_LSTM, return_sequences=True, name='lstm1'),     
            LSTM(NUM_UNITS_LSTM, return_sequences=False, name='lstm2'),
            Dense(NUM_CLASSES, activation='softmax', name='prob')
        ], name=self.name)

In [4]:
DeepConvLSTMModel().create().summary()

W0605 16:04:17.977153 139677732271872 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f08a0edfda0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 16:04:17.982750 139677732271872 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f08a0e8d2b0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "DeepConvLSTMModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1/5x1 (Conv2D)           (None, 20, 113, 64)       384       
_________________________________________________________________
conv2/5x1 (Conv2D)           (None, 16, 113, 64)       20544     
_________________________________________________________________
conv3/5x1 (Conv2D)           (None, 12, 113, 64)       20544     
_________________________________________________________________
conv4/5x1 (Conv2D)           (None, 8, 113, 64)        20544     
_________________________________________________________________
reshape (Reshape)            (None, 8, 7232)           0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 8, 128)            3768832   
_________________________________________________________________
lstm2 (LSTM)                 (None, 128)         

In [5]:
perf = []
estimate(DeepConvLSTMModel(), perf, perf)

W0605 16:04:28.019119 139677732271872 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f089862f908>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0605 16:04:28.022856 139677732271872 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f089862fe10>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(24, 113, 1)",0.000000,0,2712,0.000000,0.010848
1,conv1/5x1,"(20, 113, 64)",0.000723,384,289280,0.001536,1.157120
2,conv2/5x1,"(16, 113, 64)",0.037028,20544,231424,0.082176,0.925696
3,conv3/5x1,"(12, 113, 64)",0.027771,20544,173568,0.082176,0.694272
4,conv4/5x1,"(8, 113, 64)",0.018514,20544,115712,0.082176,0.462848
5,reshape_1,"(8, 7232)",0.000000,0,57856,0.000000,0.231424
6,lstm1 (LSTM),"(8, 128)",0.030147,3768832,14336,15.075328,0.057344
7,lstm2 (LSTM),"(128,)",0.001049,131584,14336,0.526336,0.057344
8,prob,"(18,)",0.000002,2322,36,0.009288,0.000144
9,TOTAL,"(18,)",0.115233,3964754,899260,15.859016,3.597040


In [6]:
printable_dataframe(perf, ignore_phase=False)

,Model,Phase,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,DeepConvLSTMModel,inference,"(24, 113, 1)",3964754,15.859016,0.115233,3.59704
1,DeepConvLSTMModel,training,"(24, 113, 1)",3964754,15.859016,0.345700,7.19408
